# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

In [1]:
resource_group = "fbakscpurg" # Feel free to modify these
aks_name = "fbAKSClustercpu"
location = "eastus"

image_name = "fboylu/kerasnasl-cpu" 
selected_subscription = "'Team Danielle Internal'"

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [2]:
!az login -o table

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FYRX4L92B to authenticate.
CloudName    IsDefault    Name                           State    TenantId
-----------  -----------  -----------------------------  -------  ------------------------------------
AzureCloud   False        Boston DS Dev                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Internal Consumption           Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   True         Team Danielle Internal         Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Core-ES-BLD                    Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Boston Engineering             Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        ADLTrainingMS                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Ads Eng Big Data Subscription  Enabled  72f988bf-86

In [7]:
!az account set --subscription $selected_subscription

In [3]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "fboylu@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [9]:
!az provider register -n Microsoft.ContainerService

In [3]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [4]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbakscpurg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbakscpurg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier.

In [6]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 5 --generate-ssh-keys -s Standard_D4_v2

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 5,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_D4_v2",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbakscpurg-edf507",
  "fqdn": "fbaksclust-fbakscpurg-edf507-74a0b7ae.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbakscpurg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustercpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpI

In [5]:
# !az aks scale --resource-group=$resource_group --name=$aks_name --node-count 5

In [4]:
!az aks list -o table

Name             Location    ResourceGroup    KubernetesVersion    ProvisioningState    Fqdn
---------------  ----------  ---------------  -------------------  -------------------  ----------------------------------------------------------
dangdevopsclstr  eastus      dangdevops       1.9.6                Succeeded            devops-3b040ce9.hcp.eastus.azmk8s.io
ddaks            eastus      ddaksdeploy      1.8.11               Failed               ddaks-ddaksdeploy-edf507-eefe283f.hcp.eastus.azmk8s.io
fbAKSClustercpu  eastus      fbakscpurg       1.9.6                Succeeded            fbaksclust-fbakscpurg-edf507-74a0b7ae.hcp.eastus.azmk8s.io
fbAKSClustergpu  eastus      fbaksrg          1.9.6                Succeeded            fbaksclust-fbaksrg-edf507-e12d7f40.hcp.eastus.azmk8s.io
msaksgpu         eastus      msaksrg          1.9.6                Succeeded            msaksgpu-msaksrg-edf507-0eeb29a6.hcp.eastus.azmk8s.io


### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [5]:
!sudo -i az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [6]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSClustercpu" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [7]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-1   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-2   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-3   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-4   Ready     agent     6d        v1.9.6


Let's check the pods on our cluster.

In [11]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          6d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          6d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          6d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          6d
kube-system   kube-proxy-225hz                        1/1       Running   0          6d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          6d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          6d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          6d
kube-system   kube-proxy-tvnpv                        1/1       Running   0          6d
kube-system   kube-svc-redirect-278jk                 1/1       Running   0          6d
kube-system   kube-svc-redirect

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [12]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/kerasnasl-cpu",

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [13]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [14]:
write_json_to_file(app_template, 'az-dl.json')

In [15]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [16]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "fboylu/kerasnasl-cpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
         

Next, we will use kubectl create command to deploy our application.

In [17]:
!kubectl create -f az-dl.json

deployment.apps "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [24]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-7fdd7b85d5-b8lgp               1/1       Running   0          11m
default       azure-dl-7fdd7b85d5-lk2m7               1/1       Running   0          11m
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          6d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          6d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          6d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          6d
kube-system   kube-proxy-225hz                        1/1       Running   0          6d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          6d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          6d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          6d
kube-system   kube-proxy-tvnp

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [25]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                       SOURCE                              MESSAGE
13m         13m          1         azure-dl-65485c56d8-2pl5h.153858c2a71db95e   Pod          spec.containers{azure-dl}   Normal    Killing                      kubelet, aks-nodepool1-13347348-1   Killing container with id docker://azure-dl:Need to kill Pod
13m         13m          1         azure-dl-65485c56d8-4klvn.153858c2a4284308   Pod          spec.containers{azure-dl}   Normal    Killing                      kubelet, aks-nodepool1-13347348-4   Killing container with id docker://azure-dl:Need to kill Pod
13m         13m          1         azure-dl-65485c56d8-5fkw5.153858c2aadffb8b   Pod          spec.containers{azure-dl}   Normal    Killing                      kubelet, aks-nodepool1-13347348-0   Killing container with id docker://azure-dl:Need to kill Pod
13m         13m          

Check the logs for the application pod.

In [26]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [27]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-06-15 13:38:34,051 CRIT Supervisor running as root (no user in config file)
2018-06-15 13:38:34,055 INFO supervisord started with pid 1
2018-06-15 13:38:35,057 INFO spawned: 'program_exit' with pid 11
2018-06-15 13:38:35,059 INFO spawned: 'nginx' with pid 12
2018-06-15 13:38:35,061 INFO spawned: 'gunicorn' with pid 13
2018-06-15 13:38:36,175 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-06-15 13:38:36.894701: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-06-15 13:38:40,899 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
2018-06-15 13:38:55,920 INFO success: gunicorn entered RUNNING state, process has stayed up for > than 20 seconds (startsecs)

359751680/359746192 [==============================] - 3s 0us/step
{"timestamp": "2018-06-15T13:39:37.707144Z", "logger": "

In [31]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   2         2         2            2           13m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [49]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.157.201   40.117.209.80   80:32576/TCP   44m


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources.

# Scaling

In [37]:
#!az aks scale --resource-group=$resource_group --name=$aks_name --node-count 3

In [75]:
# !kubectl describe nodes

In [45]:
!kubectl scale --current-replicas=30 --replicas=35 deployment/azure-dl

deployment.extensions "azure-dl" scaled


In [50]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-7fdd7b85d5-22996               1/1       Running   0          32m
default       azure-dl-7fdd7b85d5-4p5m9               1/1       Running   0          22m
default       azure-dl-7fdd7b85d5-6l7rp               1/1       Running   0          32m
default       azure-dl-7fdd7b85d5-7w26q               1/1       Running   0          22m
default       azure-dl-7fdd7b85d5-8n6vm               1/1       Running   0          26m
default       azure-dl-7fdd7b85d5-98rvv               1/1       Running   0          22m
default       azure-dl-7fdd7b85d5-9dss5               1/1       Running   0          8m
default       azure-dl-7fdd7b85d5-9k5zp               1/1       Running   0          26m
default       azure-dl-7fdd7b85d5-9zd2f               1/1       Running   0          26m
default       azure-dl-7fdd7b85d5-b8lgp               1/1       Running   0          51m
default       azure-dl

In [51]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   35        35        35           35          51m


In [30]:
!kubectl logs azure-dl-7fdd7b85d5-lk2m7

2018-06-15 13:38:43,377 CRIT Supervisor running as root (no user in config file)
2018-06-15 13:38:43,379 INFO supervisord started with pid 1
2018-06-15 13:38:44,382 INFO spawned: 'program_exit' with pid 10
2018-06-15 13:38:44,384 INFO spawned: 'nginx' with pid 11
2018-06-15 13:38:44,385 INFO spawned: 'gunicorn' with pid 12
2018-06-15 13:38:45,425 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-06-15 13:38:45.976123: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-06-15 13:38:49,982 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
2018-06-15 13:39:04,999 INFO success: gunicorn entered RUNNING state, process has stayed up for > than 20 seconds (startsecs)

359751680/359746192 [==============================] - 3s 0us/step
{"timestamp": "2018-06-15T13:39:43.549679Z", "host": "az

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [9]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [55]:
!az aks delete -n $aks_name -g $resource_group -y

inished ..

In [56]:
!az group delete --name $resource_group -y

inished ..